In [ ]:
import torch
from torch.utils.benchmark import Timer

# Assuming you have a model and input tensor
model = torch.load('your_model.pth')  # Load your model
input_tensor = torch.randn(1, 3, 224, 224)  # Example input tensor

# Set the model to evaluation mode
model.eval()

# Create a timer
timer = Timer()

# Measure inference time
with torch.no_grad():
    timer.start()
    output = model(input_tensor)
    timer.stop()

## Intro
#### Background - Recap
The goal of this work was to evaluate different models for Iris segmentation, as part of the computer vision stack of a robotic cataract system developed by Forsight Robotics. The requirements from this model are as following:

* Recieve full-HD, stereo, coaxial video of the surgical field, including: the eye, tools, speculum and other obstructions. 
* Segment the *limbus* of the eye - the bor
* Segmentation must be robust to overcome obstructions due to tools, changes in illumination, motion of the anatomy during surgery, and reflections.
* The model should be computationally inexpensive to allow real-time inference at 60 Hz.
  
<img src="report\eye_anatomy.jpeg" alt="Eye" style="width: 25%"/>

The background and clinical need is explained in more detail in the mid-project report.

#### Project Overview



## Datasets
#### Training Dataset


#### Evaluation Dataset



## Code explanation
Th1

## Transforms Evaluation
`Transforms_v0` vs `Transforms_v1` using Unet with ResNet 34 encoder.

Average metrics

## Loss Evaluation
`DiceLoss` vs `EdgeDice` using Unet with ResNet 34 encoder.

Average metrics

## Model Evaluation

This notebook will evaluate models trained from the previous parts.


### 1. Training convergance

We will look at the loss of the model vs. epoch to determine how many epochs (more or less) are needed for training. Unet with ResNet34 pre-trained encoder.

Histogram

Generalization on test images

### 3. Effect of loss function


Histogram

#### Interim conclusions
* what loss
* transform?

### 4. Model comparison
Comapre models: 
* inference time on test images
* histograms
* metrics

#### Qualitative comparison111